<header>
    <h1>Audio Classifier Python</h1>
    <h2>Marco Bondaschi, Mauro Conte</h2>
    <h3>Marco@Bondaschi, Mauro@Conte</h3>
    <h4>Laboratorio di Telecomunicazioni</h4>
    <h5>Università Degli Studi Di Brescia</h5>
    <h6>13/03/2017</h6>
</header>
<hr>
<section>
    <h3>Classificazioni</h3>
    <ol>
        <li>Mth1: una pase perogni specie, 
        proietto ricostruisco e aluto l'errore</li>
        <li>Mth2: bla bla bla</li>
        <li>Mth3:bla bla</li>
    </ol>
</section>

# Questo è header
## subtitle
=============
# sottoheader
## sottosottotitolo
_____________

In [ ]:
from scipy import signal, linalg
import scipy.io.wavfile
import numpy as np
import o
import math
import sys

def load_audio(path):
    audio_list = []
    for filename in os.listdir(path):
        rate, data = scipy.io.wavfile.read(path + filename) 
        audio_list.append((rate, data, filename))
    return audio_list


def get_spectrogram(data, rate):
    f, t, sxx = signal.spectrogram(data, rate, 'hamming', 512, 160, 512)
    spectrogram = 10 * np.log10(sxx + sys.float_info.min)
    return spectrogram


def get_spectrogram_from_list(collection):
    spectrogram = None
    for rate,data,f_name in collection:
        if spectrogram is None: spectrogram = get_spectrogram(data,rate)
        else: scipy.hstack((spectrogram, get_spectrogram(data,rate)))
    return spectrogram

def get_base_from_spectrogram(spectrogram,percent):
    u, s, v = np.linalg.svd(spectrogram)
    energy_e_value = np.sum(s)
    current_sum = 0
    percent_sum = energy_e_value * percent / 100
    for i in range(len(s)):
        current_sum += s[i]
        if current_sum >= percent_sum:
            break
    base = u[:, range(0, i)]
    return base


def get_feature_mean(spectrogram, base):
    mean_vector = np.zeros(base.shape[1])
    coefficient_on_base = np.dot(spectrogram.T, base)
    mean_vector += np.mean(coefficient_on_base, 0)
    return mean_vector

def get_feature_mean_from_list(collection, base):
    mean_vector = np.zeros(base.shape[1])
    for rate, data, filename in collection:
        spectrogram = get_spectrogram(data,rate)
        mean_vector += get_feature_mean(spectrogram,base)
    return mean_vector / len(collection)

def get_error_base(spectrogram, base):
    coefficient_on_base = np.dot(spectrogram.T, base)
    spectrogram_ric = np.dot(coefficient_on_base, base.T).T
    error = np.linalg.norm(spectrogram - spectrogram_ric)
    return error

In [195]:
path_db = os.getcwd()+'/05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/'
path_music = path_db + 'music/'
path_speech = path_db + 'speech/'
path_unknowns = path_db.replace('database/', '') + 'unknownSounds/'

In [196]:
music = load_audio(path_music)
speech = load_audio(path_speech)
unknowns = load_audio(path_unknowns)

In [197]:
spectrogram_music = get_spectrogram_from_list(music)
spectrogram_speech = get_spectrogram_from_list(speech)
spectrogram_total = scipy.hstack((spectrogram_music, spectrogram_speech))

In [213]:
energy = 50
base_music = get_base_from_spectrogram(spectrogram_music,energy)
base_speech = get_base_from_spectrogram(spectrogram_speech,energy)
base_total = get_base_from_spectrogram(spectrogram_total,energy)

In [214]:
feature_mean_music = get_feature_mean_from_list(music,base_music)
feature_mean_speech = get_feature_mean_from_list(speech,base_speech)
feature_mean_music_on_total = get_feature_mean_from_list(music,base_total)
feature_mean_speech_on_total = get_feature_mean_from_list(speech,base_total)

In [216]:
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = get_error_base(current_spectrogram,base_speech)
    err_on_music = get_error_base(current_spectrogram,base_music)
    print('\tMetodo1:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

    err_on_speech = np.linalg.norm(feature_mean_speech-get_feature_mean(current_spectrogram,base_speech))
    err_on_music = np.linalg.norm(feature_mean_music-get_feature_mean(current_spectrogram,base_music))
    print('\tMetodo2:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')
    
    err_on_speech = np.linalg.norm(feature_mean_speech_on_total-get_feature_mean(current_spectrogram,base_total))
    err_on_music = np.linalg.norm(feature_mean_music_on_total-get_feature_mean(current_spectrogram,base_total))
    print('\tMetodo3:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo1::  PARLATO
	Metodo2::  MUSICA
	Metodo3::  MUSICA
Item:  007si1079.wav
	Metodo1::  PARLATO
	Metodo2::  PARLATO
	Metodo3::  PARLATO
Item:  007si1271.wav
	Metodo1::  PARLATO
	Metodo2::  PARLATO
	Metodo3::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo1::  MUSICA
	Metodo2::  MUSICA
	Metodo3::  MUSICA
Item:  014si1291.wav
	Metodo1::  PARLATO
	Metodo2::  PARLATO
	Metodo3::  PARLATO
Item:  016si1621.wav
	Metodo1::  PARLATO
	Metodo2::  PARLATO
	Metodo3::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo1::  MUSICA
	Metodo2::  MUSICA
	Metodo3::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo1::  MUSICA
	Metodo2::  MUSICA
	Metodo3::  MUSICA
